<a href="https://colab.research.google.com/github/NongNNew/Project_499/blob/main/Project_499_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN

In [1]:
!git clone https://github.com/NongNNew/Project_499.git

Cloning into 'Project_499'...
remote: Enumerating objects: 1006, done.
remote: Counting objects: 100% (174/174), done.
remote: Compressing objects: 100% (169/169), done.
remote: Total 1006 (delta 13), reused 166 (delta 5), pack-reused 832
Receiving objects: 100% (1006/1006), 167.08 MiB | 20.47 MiB/s, done.
Resolving deltas: 100% (90/90), done.
Checking out files: 100% (802/802), done.


In [2]:
fruit_veget = ['กระชาย','กระท้อน','กระเทียม1','กระเทียม2','กระเพรา','กล้วยน้ำว้า','กล้วยหอม',
               'ข้าวโพด','ไข่น้ำ',
               'ตะขบไทย','ตะไคร้',
               'ถั่วฝักยาว','ถั่วลันเตา','ถั่วลิสง',
               'ทับทิม',
               'น้อยหน่า','น้ำเต้า',
               'ผักกระเฉด','ผักกุยช่าย','ผักขึ้นช่าย','ผักชะอม','ผักชี','ผักชีฝรั่ง','ผักตำลึง',
               'บวบ','ใบชะพลู1','ใบชะพลู2','ใบบัวบก','ใบแมงลัก','ใบยอ',
               'พริกขี้หนู','พริกสด','พุทรา','เพกา',
               'ฝรั่ง',
               'ฟัก','ฟักทอง',
               'มะปราง','มะพลับ','มะละกอ',
               'สับปะรด']

source = {'audio_time':[],
          'sampling_rate':[],
          'feature_extraction':[],
          'label':[],
          'description':[]}

In [3]:
import pandas as pd
from glob import glob
import librosa

for types in fruit_veget:
    audio_files = glob('/content/Project_499/Record_audio/' + str(types) + '/*.wav')

    for audio in audio_files:
        y,sr = librosa.load(audio,duration=6,offset=0)
        source['description'].append(str(types))
        source['audio_time'].append(y)
        source['sampling_rate'].append(sr)
        source['feature_extraction'].append(librosa.feature.mfcc(y,sr,n_mfcc=13))
        source['label'].append(fruit_veget.index(types))

In [4]:
df = pd.DataFrame.from_dict(source)
df

,audio_time,sampling_rate,feature_extraction,label,description
0,"[0.0, 0.0, 0.0, 3.0517578e-05, -3.0517578e-05,...",22050,"[[-648.3979, -649.9498, -650.841, -648.4261, -...",0,กระชาย
1,"[-0.00021362305, -0.00024414062, -0.0002593994...",22050,"[[-662.2338, -663.28766, -665.21606, -668.8302...",0,กระชาย
2,"[-0.0001373291, -0.00018310547, -4.5776367e-05...",22050,"[[-655.651, -653.5817, -654.66907, -655.0482, ...",0,กระชาย
3,"[1.5258789e-05, -4.5776367e-05, 0.0, 1.5258789...",22050,"[[-671.40265, -672.8409, -673.3839, -673.3374,...",0,กระชาย
4,"[-0.0002593994, -0.00021362305, -0.0002593994,...",22050,"[[-677.20435, -674.49866, -673.315, -679.4692,...",0,กระชาย
...,...,...,...,...,...
795,"[0.0, 0.0, 1.5258789e-05, 0.0, 3.0517578e-05, ...",22050,"[[-665.2395, -667.56616, -666.0286, -661.08685...",40,สับปะรด
796,"[-4.5776367e-05, -7.6293945e-05, -0.0001525878...",22050,"[[-642.5843, -642.5421, -642.17224, -642.6295,...",40,สับปะรด
797,"[-0.00018310547, -0.0002593994, -0.00022888184...",22050,"[[-662.2502, -662.54706, -661.8858, -660.38995...",40,สับปะรด
798,"[-3.0517578e-05, -1.5258789e-05, -0.0001983642...",22050,"[[-664.07837, -662.51434, -660.61566, -660.430...",40,สับปะรด


In [11]:
import numpy as np
import pandas as pd

Test = pd.DataFrame(columns=source.keys())

# create Test model
for description in np.unique(df['description']):
    imp = df[df['description']==description].sample(5,random_state=1)
    Test = pd.concat([Test,imp], axis=0)

# create Train model
Train = df.drop(Test.index)

X_train, X_test = np.array(Train['feature_extraction'].to_list()), np.array(Test['feature_extraction'].to_list())
y_train, y_test = np.array(Train['label'].to_list()), np.array(Test['label'].to_list())

In [13]:
import tensorflow as tf
from tensorflow import keras 
np.random.seed(42)
tf.random.set_seed(42)

model = keras.Sequential()
model.add(keras.layers.Conv2D(128, 5, input_shape=(13, 259, 1), activation='relu'))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.MaxPool2D())
model.add(keras.layers.Conv2D(128, 5, activation='relu',padding='same'))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.MaxPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.Dense(len(np.unique(source['label'])), activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train[:,:,:,None], y_train, epochs=100)

Epoch 1/100
19/19 [==============================] - 1s 38ms/step - loss: 22.2726 - accuracy: 0.0235
Epoch 2/100
19/19 [==============================] - 1s 33ms/step - loss: 3.5415 - accuracy: 0.0723
Epoch 3/100
19/19 [==============================] - 1s 33ms/step - loss: 3.0502 - accuracy: 0.1378
Epoch 4/100
19/19 [==============================] - 1s 32ms/step - loss: 2.3891 - accuracy: 0.3261
Epoch 5/100
19/19 [==============================] - 1s 32ms/step - loss: 1.8108 - accuracy: 0.4958
Epoch 6/100
19/19 [==============================] - 1s 33ms/step - loss: 1.4509 - accuracy: 0.5815
Epoch 7/100
19/19 [==============================] - 1s 32ms/step - loss: 1.0675 - accuracy: 0.6790
Epoch 8/100
19/19 [==============================] - 1s 32ms/step - loss: 0.8267 - accuracy: 0.7479
Epoch 9/100
19/19 [==============================] - 1s 33ms/step - loss: 0.6761 - accuracy: 0.7714
Epoch 10/100
19/19 [==============================] - 1s 32ms/step - loss: 0.6361 - accuracy: 0.815

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 9, 255, 128)       3328      
_________________________________________________________________
dropout_3 (Dropout)          (None, 9, 255, 128)       0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 127, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 127, 128)       409728    
_________________________________________________________________
dropout_4 (Dropout)          (None, 4, 127, 128)       0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 2, 63, 128)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 16128)            

In [15]:
model.evaluate(X_test[:,:,:,None],y_test)

7/7 [==============================] - 0s 38ms/step - loss: 0.1336 - accuracy: 0.9561


[0.1335848867893219, 0.9560975432395935]

In [16]:
from sklearn.metrics import classification_report

predicted_classes = np.argmax(np.round(model.predict(X_test[:,:,:,None])),axis=1)
target_names = [f"Class {label}" for label in range(len(np.unique(source['label'])))]
print(classification_report(y_test, predicted_classes, target_names=target_names))

              precision    recall  f1-score   support

     Class 0       0.62      1.00      0.77         5
     Class 1       0.67      0.80      0.73         5
     Class 2       1.00      1.00      1.00         5
     Class 3       1.00      1.00      1.00         5
     Class 4       1.00      1.00      1.00         5
     Class 5       1.00      1.00      1.00         5
     Class 6       1.00      1.00      1.00         5
     Class 7       1.00      0.80      0.89         5
     Class 8       1.00      0.80      0.89         5
     Class 9       1.00      0.80      0.89         5
    Class 10       1.00      1.00      1.00         5
    Class 11       1.00      1.00      1.00         5
    Class 12       1.00      0.80      0.89         5
    Class 13       1.00      1.00      1.00         5
    Class 14       1.00      1.00      1.00         5
    Class 15       0.80      0.80      0.80         5
    Class 16       1.00      1.00      1.00         5
    Class 17       1.00    